In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
%matplotlib inline

import os
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

import numpy as np

# Sort files in folder
from natsort import natsorted, ns

Mounted at /content/drive/


In [2]:
characters = ['0','1','2','3','4','5','6','7','8','9','-','+','*']
data_train_path = []
data_test_path =[]
for ch in characters:
  data_train_path.append(r"/content/drive/My Drive/Math_Solver/assets/pruned_dataset/" + ch + "/training/")
  data_test_path.append(r"/content/drive/My Drive/Math_Solver/assets/pruned_dataset/" + ch + "/testing/")

train_folder = []
for folder in data_train_path:
  train_folder.append([x for x in natsorted(os.listdir(folder))])
test_folder = []
for folder in data_test_path:
  test_folder.append([x for x in natsorted(os.listdir(folder))])

def readIMGFiles(Folder,p): # Folder, p-path
  files = np.zeros((len(Folder), 32, 32), dtype=np.uint8)
  for i in range(len(Folder)):
    img = imread(p+Folder[i])[:,:]
    img = resize(img, (32, 32), mode='constant', preserve_range=True)
    files[i] = img.reshape(32,32)
  return files


## array reference key for paths
## 0 - 9 : numbers 0-9
## 10 : subtraction
## 11 : addition
## 12 : multiplication
## 13 : open parenthesis
## 14 : close parenthesis
## 15 : y-variable
## 16 : equal sign

In [ ]:
training_data = readIMGFiles(train_folder[0],data_train_path[0])
testing_data = readIMGFiles(test_folder[0],data_test_path[0])
for i in range(1,13):
  training_data = np.concatenate((training_data,readIMGFiles(train_folder[i], data_train_path[i])))
  testing_data = np.concatenate((testing_data,readIMGFiles(test_folder[i], data_test_path[i])))
training_data = training_data[...,np.newaxis]
testing_data = testing_data[...,np.newaxis]
print(training_data.shape)

In [ ]:
training_labels = np.zeros([750*13])
testing_labels = np.zeros([150*13])

j = 1
for i in range(750*13):
  if i >= j * 750:
    j += 1
  training_labels[i] = int(j - 1)
training_labels = training_labels[...,np.newaxis]

j = 1
for i in range(150*13):
  if i >= j * 150:
    j += 1
  testing_labels[i] = int(j - 1)
testing_labels = testing_labels[...,np.newaxis]

print(training_labels.shape)
print(training_labels[749])

print(testing_labels.shape)
print(testing_labels[100])

In [ ]:
# change as fit/needed

model = models.Sequential() # creates sequential model
# below are the network layers being added
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1))) # no rgb grey black n white
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))# 17 symbols/ characters 

model.summary()

# change optimizer, loss, and metrics
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# change epochs as needed for higher accuracy
model.fit(training_data, training_labels, epochs=7, batch_size=1)

# save model
model.save('/content/drive/My Drive/Math_Solver/assets/my_model.h5')

In [ ]:
prediction = model.predict(testing_data, verbose=1)
maxElem = prediction[200]
result = np.where( maxElem == np.amax(maxElem))
print(eval(characters[result[0][0]]))